In [2]:
!pip install tensorflow
!pip install keras



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import pandas as pd
import string
import numpy as np
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import keras.utils as ku
import tensorflow as tf
# Set random seeds for reproducibility
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

# Load CSV files
df1 = pd.read_csv('USvideos.csv')
df2 = pd.read_csv('CAvideos.csv')
df3 = pd.read_csv('GBvideos.csv')

# Load JSON files containing category names
data1 = json.load(open('US_category_id.json'))
data2 = json.load(open('CA_category_id.json'))
data3 = json.load(open('GB_category_id.json'))


# Function to extract category information from JSON files
def category_extractor(data):
    i_d = [int(data['items'][i]['id']) for i in range(len(data['items']))]
    title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
    return dict(zip(i_d, title))

# Map category titles to each video in the datasets
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))




# Join the dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)

# Drop duplicate videos based on 'video_id'
df = df.drop_duplicates('video_id')

# Collect only titles of entertainment videos
entertainment = df[df['category_title'] == 'Entertainment']['title'].tolist()






# Remove punctuation and convert text to lowercase
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    return text.encode('utf8').decode('ascii', 'ignore')

corpus = [clean_text(title) for title in entertainment]




# Tokenizer for the corpus
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    
    return input_sequences, total_words



# Generate sequences and total word count
inp_sequences, total_words = get_sequence_of_tokens(corpus)

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len




# Get padded sequences
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 — LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

# Create the model
model = create_model(max_sequence_len, total_words)

# Train the model
model.fit(predictors, label, epochs=20, verbose=1)




def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

# Generate text with the trained model
print(generate_text("HAPPY", 5, model, max_sequence_len))


C:\Users\varsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 129s 56ms/step - loss: 8.2025
Epoch 2/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 139s 55ms/step - loss: 7.1033
Epoch 3/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 122s 55ms/step - loss: 6.6049
Epoch 4/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 122s 55ms/step - loss: 6.1974
Epoch 5/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 135s 52ms/step - loss: 5.8351
Epoch 6/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 120s 54ms/step - loss: 5.5246
Epoch 7/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 121s 55ms/step - loss: 5.2422
Epoch 8/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 125s 57ms/step - loss: 4.9872
Epoch 9/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 122s 55ms/step - loss: 4.7463
Epoch 10/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 121s 55ms/step - loss: 4.5275
Epoch 11/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 120s 55ms/step - loss: 4.3205
Epoch 12/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 124s 56ms/step - loss: 4.1369
Epoch 13/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 121s 55ms/step - loss: 3.9673
Epoch 14/20
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 151s 69ms/step - 